In [ ]:
import numpy as np
import math

def bit_length(x):
    return int(np.ceil(np.log2(abs(x) + 1))) if x != 0 else 1

def h_matrix(n, L):
    return np.eye(n) * (n ** 2) * (2 ** (2 * L))

def constraint_violation(center, epsilon):
    for i, (a, b) in enumerate(constraints):
        if float((a @ center)[0]) > b + epsilon:
            return i
    return None

def update(center, h, a, n):

    Ha = h @ a
    aHa = (a.T @ Ha)
    denom = np.sqrt(aHa)
    if denom < 1e-6:
        return None, None
    new_center = (center - (1 / (n + 1)) * Ha / denom)

    outer = Ha @ Ha.T
    new_h = (n**2 / (n**2 - 1)) * (h - (2 / (n + 1)) * outer / aHa)

    return new_center, new_h


def tiksloF(x):
    # Tavo uždavinio tikslo funkcija: 2x1 - 3x2 - 5x4
    return 2 * x[0] - 3 * x[1] - 5 * x[3]

def reflection(Xc, Xh, alpha):
    return Xc + alpha * (Xc - Xh)

def expansion(Xc, Xr, gamma):
    return Xc + gamma * (Xr - Xc)

def contraction(Xc, Xh, rho):
    return Xc + rho * (Xh - Xc)

def shrink(simplex, values, sigma):
    for i in range(1, len(simplex)):
        simplex[i] = simplex[0] + sigma * (simplex[i] - simplex[0])
        values[i] = tiksloF(simplex[i])

def simpleksoMetodas(pradinisTaskas):
    tikslas = 0
    alpha = 0.55
    gamma = 1.5
    rho = 0.5
    sigma = 0.5
    epsilon = 1e-6
    n = len(pradinisTaskas)

    a1 = (math.sqrt(n + 1) + n - 1) / (n * math.sqrt(2)) * alpha
    a2 = (math.sqrt(n + 1) - 1) / (n * math.sqrt(2)) * alpha

    iterations = 0
    taskai = [np.array(pradinisTaskas)]
    for i in range(1, n + 1):
        e = np.zeros(n)
        e[i - 1] = 1
        new_point = pradinisTaskas + a2 * np.ones(n) + (a1 - a2) * e
        taskai.append(new_point)

    reiksmes = [tiksloF(x) for x in taskai]
    tikslas += len(taskai)

    while True:
        iterations += 1
        # Rikiavimas pagal reikšmę
        idx = np.argsort(reiksmes)
        taskai = [taskai[i] for i in idx]
        reiksmes = [reiksmes[i] for i in idx]

        dx = np.linalg.norm(taskai[1] - taskai[0])
        if dx < epsilon:
            print("Simplekso minimumo taskas:", taskai[0])
            print("Iteraciju skaicius:", iterations)
            print("Tikslo funkcijos kvietimu:", tikslas)
            break

        Xc = np.mean(taskai[:-1], axis=0)
        Xr = reflection(Xc, taskai[-1], alpha)
        fr = tiksloF(Xr)
        tikslas += 1

        fl = reiksmes[0]
        fg = reiksmes[-2]
        fh = reiksmes[-1]

        if fr < fl:
            Xe = expansion(Xc, Xr, gamma)
            fe = tiksloF(Xe)
            tikslas += 1
            if fe < fr:
                taskai[-1] = Xe
                reiksmes[-1] = fe
            else:
                taskai[-1] = Xr
                reiksmes[-1] = fr
        elif fr < fg:
            taskai[-1] = Xr
            reiksmes[-1] = fr
        else:
            if fr < fh:
                X = contraction(Xc, Xr, rho)
            else:
                X = contraction(Xc, taskai[-1], rho)
            fx = tiksloF(X)
            tikslas += 1
            if fx < fh:
                taskai[-1] = X
                reiksmes[-1] = fx
            else:
                shrink(taskai, reiksmes, sigma)
                tikslas += 1

raw_constraints = [
    (np.array([-1, 1, -1, -1]), 8),
    (np.array([2, 4, 0, 0]), 10),
    (np.array([0, 0, 1, 1]), 3),
    (np.array([-1, 0, 0, 0]), 0),
    (np.array([0, -1, 0, 0]), 0),
    (np.array([0, 0, -1, 0]), 0),
    (np.array([0, 0, 0, -1]), 0)
]

L = max(bit_length(x) for a, b in raw_constraints for x in a.tolist() + [b])
epsilon = 2 ** (-2 * L)
constraints = [(a, b + epsilon) for a, b in raw_constraints]
n =len(constraints[0][0])
r = n * 2 ** L
K = 16 * n * (n + 1) * L
center = np.zeros((n, 1))
h = h_matrix(n, L)

c = np.array([2, -3, 0, -5]).reshape(-1, 1)

best_x = None
best_value = float('inf')

for k in range(K):

    i = constraint_violation(center, epsilon)
    if i is None:
        simpleksoMetodas(center)
        break


    a = constraints[i][0].reshape(-1, 1)    
    center, h = update(center, h, a, n)

Stabdoma – per ilgai be pagerėjimo


C:\Users\likava\AppData\Local\Temp\ipykernel_16060\636340843.py:60: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  value = float(c.T @ center)
